In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc
import numpy as np
import six
import types

from keras import backend as K
import tensorflow as tf
import LoggerYN as YN

from sklearn.model_selection import train_test_split
import unicodedata
import re
import numpy as np
import os
import time
import datetime
import sys

from data import load_ptb, load_ptb_vocab
from data import load_imdb 


warnings.filterwarnings('ignore')



Using TensorFlow backend.
C:\Users\wahab\.conda\envs\myenv\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\wahab\.conda\envs\myenv\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\wahab\.conda\envs\myenv\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\wahab\.conda\envs\myenv\lib\site-packages\tenso

In [2]:
def run_imdb(n_epochs):
    np_load_old = np.load
    np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
    
    def imdb_iterator_op(train, batch_size, vocabulary_size, seq_len):
        x, y = load_imdb(train, vocabulary_size, seq_len)
        lens = np.array([len(xi) for xi in x], dtype=int)
        x = np.array([np.pad(xi, (0, seq_len - len(xi)), 'constant') for xi in x], dtype=int)

        x_placeholder = tf.placeholder(x.dtype, x.shape)
        len_placeholder = tf.placeholder(lens.dtype, lens.shape)
        y_placeholder = tf.placeholder(y.dtype, y.shape)

        dataset = tf.data.Dataset.from_tensor_slices((x_placeholder, len_placeholder, y_placeholder))
        if train:
            dataset = dataset.shuffle(buffer_size=len(y))
        dataset = dataset.batch(batch_size)
        iterator = dataset.make_initializable_iterator()
        return iterator, {x_placeholder: x, len_placeholder: lens, y_placeholder: y}


    # In[3]:


    class ImdbLstm:

        def __init__(self, vocabulary_size, embedding_size, hidden_size):
            self.embed = tf.get_variable('embed', (vocabulary_size, embedding_size), 
                                         initializer=tf.initializers.random_uniform(-1.0, 1.0))
            self.lstm = tf.nn.rnn_cell.LSTMCell(hidden_size, initializer=lstm_kernel_initializer)
            self.fc = tf.layers.Dense(units=1)

        def __call__(self, x, lens):
            x = tf.nn.embedding_lookup(self.embed, x)
            o, (c, h) = tf.nn.dynamic_rnn(self.lstm, x, sequence_length=lens, dtype=tf.float32)   
            p = self.fc(h)
            return tf.reshape(p, [-1])


    def lstm_kernel_initializer(shape, dtype, partition_info):
        hidden_size = shape[1] // 4
        input_size = shape[0] - hidden_size
        kernel = tf.glorot_uniform_initializer(seed=None)((input_size, shape[1]))
        recurrent = tf.orthogonal_initializer(seed=None)((hidden_size, shape[1]))
        return tf.concat([kernel, recurrent], axis=0)


    # In[4]:


    def imdb_train(model, data_iterator_op, loss_op, optimizer_op, placeholders, epoch, print_every=50):
        sess = tf.get_default_session()

        data_iterator, data_iterator_feed = data_iterator_op
        sess.run(data_iterator.initializer, data_iterator_feed)

        losses = []
        i = 0
        data_op = data_iterator.get_next()
        while True:
            try:
                data = sess.run(data_op)
                feed_dict = {placeholder: value for placeholder, value in zip(placeholders, data)}
                loss, _ = sess.run([loss_op, optimizer_op], feed_dict)

                losses.append(loss)
                if (i + 1) % print_every == 0:
                    print('[%d, %5d] train loss: %.3f' % (epoch, i + 1, np.mean(losses)))
                    sys.stdout.flush()
                    losses = []
                i += 1
            except tf.errors.OutOfRangeError:
                break


    def imdb_test(model, data_iterator_op, loss_op, correct_op, placeholders, epoch):
        sess = tf.get_default_session()

        data_iterator, data_iterator_feed = data_iterator_op
        sess.run(data_iterator.initializer, data_iterator_feed)

        losses = []
        correct, total = 0, 0
        data_op = data_iterator.get_next()
        while True:
            try:
                data = sess.run(data_op)

                feed_dict = {placeholder: value for placeholder, value in zip(placeholders, data)}
                loss, preds = sess.run([loss_op, correct_op], feed_dict)

                losses.append(loss)
                correct += preds.sum()
                total += preds.shape[0]
            except tf.errors.OutOfRangeError:
                break
        print('[%d] test loss: %.3f accuracy: %.3f' % (epoch, np.mean(losses), correct / total * 100))
        sys.stdout.flush()


    def imdb_run(n_epochs, vocabulary_size, seq_len, batch_size, embedding_size, hidden_size):
        np.random.seed(1)
        tf.set_random_seed(1)

        tf.reset_default_graph()

        train_iterator_op = imdb_iterator_op(train=True, batch_size=batch_size, 
                                             vocabulary_size=vocabulary_size, seq_len=seq_len)
        test_iterator_op = imdb_iterator_op(train=False, batch_size=batch_size, 
                                             vocabulary_size=vocabulary_size, seq_len=seq_len)

        inputs = tf.placeholder(tf.int32, [None, seq_len])
        lens = tf.placeholder(tf.int32, [None])
        labels = tf.placeholder(tf.int32, [None])
        placeholders = (inputs, lens, labels)

        model = ImdbLstm(vocabulary_size, embedding_size, hidden_size)
        outputs = model(inputs, lens)
        loss = tf.losses.sigmoid_cross_entropy(labels, outputs)
        optimizer = tf.train.AdamOptimizer().minimize(loss)
        correct = tf.equal(tf.cast(outputs >= 0.0, tf.int32), labels)

        sess = tf.Session()
        sess.run(tf.global_variables_initializer())

        start = time.time()
        current_time = time.time()
        time_consumed=current_time-start
        epoch=0

        memT,cpuT,gpuT = YN.StartLogger("TensorFlow","IMDB_CPU")

        with sess.as_default():
            while (time_consumed <= 86400 and epoch <= n_epochs):
                print("\n\nEpoch ",epoch)
                imdb_train(model, train_iterator_op, loss, optimizer, placeholders, epoch)
                imdb_test(model, test_iterator_op, loss, correct, placeholders, epoch)
                epoch += 1
                time_consumed=(time.time())-start
                print("Time since beginning ", str(datetime.timedelta(seconds=time_consumed)) )
                sys.stdout.flush()


        YN.EndLogger(memT,cpuT,gpuT)
        print("\n\nTotal Time Consumed ", str(datetime.timedelta(seconds=time_consumed)))
        sys.stdout.flush()

        saver = tf.train.Saver()
        saver.save(sess, './TensorFlow_IMDB_LSTM_model')





    imdb_run(n_epochs, vocabulary_size = 5000, seq_len = 500, batch_size = 64, embedding_size = 32, hidden_size = 100)



In [3]:
def run_manythings(n_epochs):


    # Converts the unicode file to ascii
    def unicode_to_ascii(s):
        return ''.join(c for c in unicodedata.normalize('NFD', s)
            if unicodedata.category(c) != 'Mn')


    def preprocess_sentence(w):
        w = unicode_to_ascii(w.lower().strip())

        # creating a space between a word and the punctuation following it
        # eg: "he is a boy." => "he is a boy ." 
        # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
        w = re.sub(r"([?.!,¿])", r" \1 ", w)
        w = re.sub(r'[" "]+', " ", w)

        # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
        w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

        w = w.rstrip().strip()

        # adding a start and an end token to the sentence
        # so that the model know when to start and stop predicting.
        w = '<start> ' + w + ' <end>'
        return w

    # 1. Remove the accents
    # 2. Clean the sentences
    # 3. Return word pairs in the format: [ENGLISH, SPANISH]
    def create_dataset(path):
        lines = open(path, encoding='UTF-8').read().strip().split('\n')
        word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:100000]]

        return word_pairs

        # This class creates a word -> index mapping (e.g,. "dad" -> 5) and vice-versa 
    # (e.g., 5 -> "dad") for each language,
    class LanguageIndex():
        def __init__(self, lang):
            self.lang = lang
            self.word2idx = {}
            self.idx2word = {}
            self.vocab = set()

            self.create_index()

        def create_index(self):
            for phrase in self.lang:
                self.vocab.update(phrase.split(' '))

            self.vocab = sorted(self.vocab)

            self.word2idx['<pad>'] = 0
            for index, word in enumerate(self.vocab):
                self.word2idx[word] = index + 1

            for word, index in self.word2idx.items():
                self.idx2word[index] = word
    def max_length(tensor):
        return max(len(t) for t in tensor)


    def load_dataset(path):
        # creating cleaned input, output pairs
        pairs = create_dataset(path)

        # index language using the class defined above    
        inp_lang = LanguageIndex(sp for en, sp in pairs)
        targ_lang = LanguageIndex(en for en, sp in pairs)

        # Vectorize the input and target languages

        # Spanish sentences
        input_tensor = [[inp_lang.word2idx[s] for s in sp.split(' ')] for en, sp in pairs]

        # English sentences
        target_tensor = [[targ_lang.word2idx[s] for s in en.split(' ')] for en, sp in pairs]

        # Calculate max_length of input and output tensor
        # Here, we'll set those to the longest sentence in the dataset
        max_length_inp, max_length_tar = max_length(input_tensor), max_length(target_tensor)

        # Padding the input and output tensor to the maximum length
        input_tensor = tf.keras.preprocessing.sequence.pad_sequences(input_tensor, 
                                                                     maxlen=max_length_inp,
                                                                     padding='post')

        target_tensor = tf.keras.preprocessing.sequence.pad_sequences(target_tensor, 
                                                                      maxlen=max_length_tar, 
                                                                      padding='post')

        return input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_tar


    # In[3]:


    def create_db(path_to_file):
        input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_targ = load_dataset(path_to_file)
        # Creating training and validation sets using an 80-20 split
        input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2,random_state=42)
        vocab_inp_size = len(inp_lang.word2idx)
        vocab_tar_size = len(targ_lang.word2idx)
        return input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val,vocab_inp_size,vocab_tar_size,max_length_inp, max_length_targ


    # In[4]:


    class Model:
        def __init__(self, vocab_inp,vocab_tar, embedding_dim, units, batch_sz):
            self.batch_sz = batch_sz
            self.units = units
            self.embed_enc = tf.get_variable('embed_enc', (vocab_inp, embedding_dim))
            self.enc_cell =  tf.nn.rnn_cell.LSTMCell(self.units,initializer=lstm_kernel_initializer)
            self.embed_dec = tf.get_variable('embed_dec', (vocab_tar, embedding_dim))
            self.dec_cell =  tf.nn.rnn_cell.LSTMCell(self.units,initializer=lstm_kernel_initializer)
            self.fc = tf.layers.Dense(units = vocab_tar)

        def __call__(self, inp,tar):
            enc_emb = tf.nn.embedding_lookup(self.embed_enc, inp)
            _,state = tf.nn.dynamic_rnn(self.enc_cell,inputs=enc_emb,dtype=tf.float32)
            dec_emb = tf.nn.embedding_lookup(self.embed_dec, tar)
            output,_ = tf.nn.dynamic_rnn(self.enc_cell,inputs=dec_emb,dtype=tf.float32)
            preds = self.fc(output)
            return preds    
    def lstm_kernel_initializer(shape, dtype, partition_info):
        hidden_size = shape[1] // 4
        input_size = shape[0] - hidden_size
        kernel = tf.glorot_uniform_initializer(seed=None)((input_size, shape[1]))
        recurrent = tf.orthogonal_initializer(seed=None)((hidden_size, shape[1]))
        return tf.concat([kernel, recurrent], axis=0)


    # In[5]:


    def accuracy(y_true, y_pred):
        if not K.is_tensor(y_pred):
            y_pred = K.constant(y_pred)
            y_true = K.constant(y_true)
        y_true = K.cast(y_true, y_pred.dtype)
        return K.cast(K.equal(y_true, y_pred), K.floatx())

    def acc_function(real, pred):
        prediction=tf.argmax(pred,-1)
        acc_tensor=accuracy(real,prediction)
        return acc_tensor


    # In[6]:


    def acc(acc_,sess,input_tensor_train,target_tensor_train,N_BATCH,BATCH_SIZE,loss_,inpu,targe):
        total=0
        count=0
        for batch in range(N_BATCH):
            start = BATCH_SIZE*batch
            end = (BATCH_SIZE*batch) + BATCH_SIZE
            inp = input_tensor_train[start :end]
            targ = target_tensor_train[start:end]
            feed_dict={inpu:inp,targe:targ}
            acc_tensor = sess.run(acc_,feed_dict)        
            accu_i = np.mean(acc_tensor)
            total += accu_i
            count += 1
        print('Accuracy :{:.4f}'.format(total/count))
        sys.stdout.flush()


    # In[7]:


    def loss_function(real, pred):
        prediction=tf.argmax(pred,-1)
        loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred)
        return tf.reduce_mean(loss)


    # In[8]:


    def train(sess,input_tensor_train,target_tensor_train,N_BATCH,BATCH_SIZE,optimizer,loss_,inpu,targe):
        for batch in range(N_BATCH):
                start = BATCH_SIZE*batch
                end = (BATCH_SIZE*batch) + BATCH_SIZE
                inp = input_tensor_train[start :end]
                targ = target_tensor_train[start:end]
                loss,_=sess.run([loss_,optimizer],feed_dict={inpu:inp,targe:targ})
                if(batch % 300 ==0):
                    print('Batch {} Loss {:.4f}'.format(batch,loss))
                    sys.stdout.flush()


    # In[9]:


    def test(sess,input_tensor_train,target_tensor_train,N_BATCH,BATCH_SIZE,loss_,inpu,targe):
        t_loss = 0
        for batch in range(N_BATCH):
            start = BATCH_SIZE*batch
            correct, total = 0, 0
            end = (BATCH_SIZE*batch) + BATCH_SIZE
            inp = input_tensor_train[start :end]
            targ = target_tensor_train[start:end]
            feed_dict={inpu:inp,targe:targ}
            loss = sess.run(loss_,feed_dict)        
            t_loss += loss
        print('Validation Perplexity :{:.4f}'.format(np.power(2,t_loss/batch)))
        sys.stdout.flush()


    # In[10]:


    def run(epochs, BATCH_SIZE, embedding_dim, units):
        tf.set_random_seed(1)
        path_to_zip = tf.keras.utils.get_file('spa-eng.zip', origin='http://download.tensorflow.org/data/spa-eng.zip', extract=True)
        path_to_file = os.path.dirname(path_to_zip)+"/spa-eng/spa.txt"
        input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val,vocab_inp_size,vocab_tar_size,max_length_inp, max_length_targ = create_db(path_to_file)
        # Get parameters
        BUFFER_SIZE = len(input_tensor_train)
        N_BATCH = BUFFER_SIZE//BATCH_SIZE
        N_BATCH_VAL = len(input_tensor_val)//BATCH_SIZE

        train_samples = len(input_tensor_train)
        val_samples = len(input_tensor_val)
        #Loading data    
        data_train = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
        data_train = data_train.batch(BATCH_SIZE, drop_remainder=True)
        data_val = tf.data.Dataset.from_tensor_slices((input_tensor_val, target_tensor_val)).shuffle(BUFFER_SIZE)
        data_val = data_val.batch(BATCH_SIZE, drop_remainder=True)

        # Model initiation
        model = Model(vocab_inp_size,vocab_tar_size,embedding_dim,units,BATCH_SIZE)
        inpu = tf.placeholder(tf.int32, [None, max_length_inp])
        targe = tf.placeholder(tf.int32, [None, max_length_targ])

        predictions = model(inpu,targe[:,:-1])
        sess = tf.Session()

        loss_ = loss_function(targe[:,1:],predictions)
        acc_ = acc_function(targe[:,1:],predictions)

        optimizer = tf.train.AdamOptimizer(0.0001).minimize(loss_)

        correct=0;
        sess.run(tf.global_variables_initializer())

        start = time.time()
        current_time = time.time()
        time_consumed=current_time-start

        memT,cpuT,gpuT = YN.StartLogger("TensorFlow","Manythings")

        epoch=0

        while (time_consumed <= 86400 and epoch <= epochs):
            print("\n \nEpoch ",epoch,"\n")
            train(sess,input_tensor_train,target_tensor_train,N_BATCH,BATCH_SIZE,optimizer,loss_,inpu,targe)
            test(sess,input_tensor_train,target_tensor_train,N_BATCH_VAL,BATCH_SIZE,loss_,inpu,targe)
            acc(acc_,sess,input_tensor_train,target_tensor_train,N_BATCH_VAL,BATCH_SIZE,loss_,inpu,targe)
            epoch += 1
            time_consumed=(time.time())-start
            print("Time since beginning: ", str(datetime.timedelta(seconds=time_consumed)) )
            sys.stdout.flush()


        YN.EndLogger(memT,cpuT,gpuT)
        print("\n\nTotal Time Consumed: ", str(datetime.timedelta(seconds=time_consumed)))
        sys.stdout.flush()


    # In[ ]:


    run(n_epochs,BATCH_SIZE = 128, embedding_dim = 256, units = 256)

In [5]:
def run_ptb(n_epochs):
    class PtbIterator:

        def __init__(self, train, batch_size, seq_len, skip_step=5):
            self.data = load_ptb(train)
            self.batch_size = batch_size
            self.seq_len = seq_len
            self.skip_step = skip_step
            self.reset()

        def __iter__(self):
            self.reset()
            return self

        def __next__(self):
            x = np.zeros((self.batch_size, self.seq_len), dtype=np.int32)
            y = np.zeros((self.batch_size, self.seq_len), dtype=np.int32)

            for i in range(self.batch_size):
                if self.cur_idx + self.seq_len >= len(self.data):
                    raise StopIteration
                x[i, :] = self.data[self.cur_idx:self.cur_idx+self.seq_len]
                y[i, :] = self.data[self.cur_idx+1:self.cur_idx+self.seq_len+1]
                self.cur_idx += self.skip_step
            return x, y

        def reset(self):
            self.cur_idx = 0


    # In[3]:


    class PtbLstm:

        def __init__(self, vocabulary_size, hidden_size, num_layers, dropout):
            self.dropout = dropout
            self.embed = tf.get_variable('embed', (vocabulary_size, hidden_size), 
                                         initializer=tf.initializers.random_uniform(-1.0, 1.0))
            self.lstm = tf.nn.rnn_cell.MultiRNNCell([
                tf.nn.rnn_cell.LSTMCell(hidden_size, initializer=lstm_kernel_initializer) 
                for _ in range(num_layers)])
            self.fc = tf.layers.Dense(units=vocabulary_size)

        def __call__(self, x):
            x = tf.nn.embedding_lookup(self.embed, x)
            o, (c, h) = tf.nn.dynamic_rnn(self.lstm, x, dtype=tf.float32)
            o = tf.nn.dropout(o, keep_prob=self.dropout)
            f = self.fc(o)
            return f


    def lstm_kernel_initializer(shape, dtype, partition_info):
        hidden_size = shape[1] // 4
        input_size = shape[0] - hidden_size
        kernel = tf.glorot_uniform_initializer()((input_size, shape[1]))
        recurrent = tf.orthogonal_initializer()((hidden_size, shape[1]))
        return tf.concat([kernel, recurrent], axis=0)


    # In[4]:


    def ptb_train(model, data_iter, loss_op, optimizer_op, placeholders, epoch, print_every=600):
        sess = tf.get_default_session()

        losses = []

        for i, data in enumerate(data_iter):
            feed_dict = {placeholder: value for placeholder, value in zip(placeholders, data)}
            loss, _ = sess.run([loss_op, optimizer_op], feed_dict)

            losses.append(loss)
            if (i + 1) % print_every == 0:
                print('[%d, %5d] train loss: %.3f' % (epoch, i + 1, np.mean(losses)))
                losses = []
                sys.stdout.flush()


    # In[5]:


    def ptb_test(model, data_iter, loss_op,correct_op, placeholders, epoch):
        sess = tf.get_default_session()
        correct=0
        total=0
        total_2=0
        t2=0
        c2=0
        count=0
        noE=0
        losses = []
        for data in data_iter:
            feed_dict = {placeholder: value for placeholder, value in zip(placeholders, data)}
            loss, preds = sess.run([loss_op, correct_op], feed_dict)
            correct=np.sum(preds)
            total += correct
            noE=(preds.shape[0]*preds.shape[1])
            count += noE
            losses.append(loss)


        loss = np.mean(losses)
        perplexity = np.exp(loss)
        print('Accuracy :{:.4f}'.format(total/count))
        print('Test loss :{:.4f}'.format(loss))
        print('Perplexity :{:.4f}'.format(perplexity))
        sys.stdout.flush()    


    # In[6]:



    def ptb_run(n_epochs, hidden_size, batch_size, seq_len, dropout, num_layers):
        np.random.seed(1)
        tf.set_random_seed(1)


        ptb_vocab = load_ptb_vocab()
        vocabulary_size = len(ptb_vocab)

        tf.reset_default_graph()

        train_iter = PtbIterator(train=True, batch_size=batch_size, seq_len=seq_len)
        test_iter = PtbIterator(train=False, batch_size=batch_size, seq_len=seq_len)

        inputs = tf.placeholder(tf.int32, [batch_size, seq_len])
        labels = tf.placeholder(tf.int32, [batch_size, seq_len])
        placeholders = (inputs, labels)

        model = PtbLstm(vocabulary_size, hidden_size, num_layers, dropout)

        #Assigning outputs and casting them

        outputs = model(inputs)
        outputs_acc = tf.argmax(input = outputs,axis=-1)
        labels_acc = tf.cast(labels, tf.int64)

        #Maximising Arg

        correct = tf.equal(outputs_acc,labels_acc)


        loss = tf.losses.sparse_softmax_cross_entropy(labels, outputs)
        optimizer = tf.train.AdadeltaOptimizer(learning_rate=1.0, epsilon=1e-6).minimize(loss)

        sess = tf.Session()
        sess.run(tf.global_variables_initializer())

        start = time.time()
        current_time = time.time()
        time_consumed=current_time-start


        memT,cpuT,gpuT = YN.StartLogger("TensorFlow_CPU", "PTB")

        epoch=0

        with sess.as_default():
            while (time_consumed <= 86400 and epoch <= n_epochs):

                ptb_train(model, train_iter, loss, optimizer, placeholders, epoch)
                ptb_test(model, test_iter, loss, correct, placeholders, epoch)
                epoch += 1
                time_consumed=(time.time())-start
                print("Time since beginning: ", str(datetime.timedelta(seconds=time_consumed)) )
                sys.stdout.flush()

            end = time.time()
            YN.EndLogger(memT,cpuT,gpuT)
            print("\n\nTotal Time Consumed: ", str(datetime.timedelta(seconds=end-start)))
            sys.stdout.flush()
            saver = tf.train.Saver()
            saver.save(sess, './TensorFlow_IMDB_LSTM_model')



    # In[7]:


    ptb_run(n_epochs, hidden_size = 200, batch_size = 20, seq_len = 30, dropout = 0.5, num_layers = 2)


    # In[ ]:

In [6]:
#run_imdb(n_epochs=50)
#run_manythings(n_epochs=100)
#run_ptb(n_epochs=50)

Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
[0,   600] train loss: 6.815
[0,  1200] train loss: 6.490
[0,  1800] train loss: 6.433


KeyboardInterrupt: 